In [8]:
import math
import random

class MultiArmBanditGame:
    def __init__(self, num_actions):
        self.num_actions = num_actions
        self.action_rewards = [random.uniform(0, 1) for _ in range(num_actions)]  # Hidden reward probabilities
        self.action_counts = [0] * num_actions  # Times each action has been selected
        self.total_rewards = [0] * num_actions  # Cumulative rewards for each action

    def play_action(self, action):
        """
        Simulate playing an action. Returns reward (1 or 0) based on the hidden reward probability.
        """
        reward = 1 if random.random() < self.action_rewards[action] else 0
        self.action_counts[action] += 1
        self.total_rewards[action] += reward
        return reward

    def ucb_select_action(self, total_steps):
        """
        Use the Upper Confidence Bound algorithm to select the next action.
        """
        ucb_values = []
        for action in range(self.num_actions):
            if self.action_counts[action] == 0:
                # If the action has never been selected, assign a very high UCB value to ensure it gets explored.
                ucb_values.append(float('inf'))
            else:
                avg_reward = self.total_rewards[action] / self.action_counts[action]
                confidence_interval = math.sqrt((2 * math.log(total_steps)) / self.action_counts[action])
                ucb_values.append(avg_reward + confidence_interval)

        # Choose the action with the highest UCB value
        return ucb_values.index(max(ucb_values))

    def run_game(self, total_steps):
        """
        Run the game for a specified number of steps, optimizing actions using UCB.
        """
        total_reward = 0

        for step in range(1, total_steps + 1):
            action = self.ucb_select_action(step)
            reward = self.play_action(action)
            total_reward += reward

            print(f"Step {step}: Action {action}, Reward {reward}, Total Reward: {total_reward}")

        print("\nGame Over!\n")
        print(f"Final Total Reward: {total_reward}")
        print(f"Action Selection Counts: {self.action_counts}")
        print(f"Hidden Reward Probabilities: {self.action_rewards}")

# Define the number of actions and total steps for the game
num_actions = 5
steps = 10

# Initialize and run the game
game = MultiArmBanditGame(num_actions)
game.run_game(steps)


Step 1: Action 0, Reward 1, Total Reward: 1
Step 2: Action 1, Reward 0, Total Reward: 1
Step 3: Action 2, Reward 0, Total Reward: 1
Step 4: Action 3, Reward 0, Total Reward: 1
Step 5: Action 4, Reward 0, Total Reward: 1
Step 6: Action 0, Reward 1, Total Reward: 2
Step 7: Action 0, Reward 0, Total Reward: 2
Step 8: Action 1, Reward 0, Total Reward: 2
Step 9: Action 2, Reward 0, Total Reward: 2
Step 10: Action 3, Reward 1, Total Reward: 3

Game Over!

Final Total Reward: 3
Action Selection Counts: [3, 2, 2, 2, 1]
Hidden Reward Probabilities: [0.5496604511418426, 0.3517285303348975, 0.2784178541410791, 0.6055576034262331, 0.5811879858410218]


In [10]:
import numpy as np
import math

class IoTDeviceManager:
    def __init__(self, n_devices, n_modes):
        self.n_devices = n_devices
        self.n_modes = n_modes
        self.device_efficiencies = np.zeros((n_devices, n_modes))  # Estimated efficiency for each device-mode combination
        self.device_mode_counts = np.zeros((n_devices, n_modes))  # Number of times each device-mode combination was used
        self.total_count = 0  # Total number of decisions made

    def select_device_mode(self):
        """
        Selects a device and mode using the Upper-Confidence Bound (UCB) algorithm.
        """
        if self.total_count < self.n_devices * self.n_modes:
            # Explore each device-mode combination at least once initially
            return divmod(self.total_count, self.n_modes)

        # Calculate UCB for each device-mode combination
        ucb_values = [
            self.device_efficiencies[device, mode] + math.sqrt(
                2 * math.log(self.total_count) / (self.device_mode_counts[device, mode])
            )
            for device in range(self.n_devices)
            for mode in range(self.n_modes)
        ]
        best_index = np.argmax(ucb_values)
        return divmod(best_index, self.n_modes)

    def update_device_mode_efficiency(self, device, mode, efficiency):
        """
        Updates the estimated efficiency of the chosen device-mode combination based on observed performance.
        """
        self.total_count += 1
        self.device_mode_counts[device, mode] += 1
        # Update the efficiency estimate using incremental formula
        self.device_efficiencies[device, mode] += (
            (efficiency - self.device_efficiencies[device, mode]) / self.device_mode_counts[device, mode]
        )

# Simulation setup
n_devices = 5  # Number of IoT devices
n_modes = 3  # Number of modes per device

def simulate_real_time_efficiency(device, mode, temperature, occupancy):
    """
    Simulates the real-time efficiency of a device and mode based on environmental factors.
    Returns an efficiency value between 0 and 1.
    """
    base_efficiencies = [
        [0.8, 0.7, 0.6],  # Modes for device 1
        [0.6, 0.5, 0.4],  # Modes for device 2
        [0.7, 0.65, 0.5], # Modes for device 3
        [0.5, 0.4, 0.3],  # Modes for device 4
        [0.9, 0.85, 0.8], # Modes for device 5
    ]
    temp_factor = max(0, 1 - abs(temperature - 25) / 25)  # Efficiency drops with temperature deviation
    occupancy_factor = min(1, occupancy / 10)  # Higher occupancy improves efficiency up to a threshold
    return base_efficiencies[device][mode] * temp_factor * occupancy_factor

n_rounds = 1000  # Number of optimization rounds

def optimize_energy_usage():
    manager = IoTDeviceManager(n_devices, n_modes)
    total_efficiency = 0

    for _ in range(n_rounds):
        # Simulate environmental factors
        temperature = np.random.uniform(15, 35)  # Temperature in degrees Celsius
        occupancy = np.random.randint(1, 20)  # Number of people in the house

        device, mode = manager.select_device_mode()
        efficiency = simulate_real_time_efficiency(device, mode, temperature, occupancy)
        manager.update_device_mode_efficiency(device, mode, efficiency)
        total_efficiency += efficiency

    print("Final estimated device-mode efficiencies:")
    print(manager.device_efficiencies)
    print("Device-mode selection counts:")
    print(manager.device_mode_counts)
    print("Total energy efficiency achieved:", total_efficiency)

optimize_energy_usage()


Final estimated device-mode efficiencies:
[[0.4910377  0.39457752 0.35506539]
 [0.38141063 0.32460562 0.26359294]
 [0.41758246 0.39541809 0.29081847]
 [0.33243354 0.25527678 0.19568409]
 [0.55899339 0.51883569 0.4914287 ]]
Device-mode selection counts:
[[ 97.  62.  53.]
 [ 58.  47.  38.]
 [ 69.  63.  43.]
 [ 48.  37.  31.]
 [143. 114.  97.]]
Total energy efficiency achieved: 422.75762888380206


In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = r"/content/simplified_chess_dataset.csv"
dataset = pd.read_csv(file_path)

# Prepare the data
X = dataset[[f"cell_{i}" for i in range(16)]]
y = dataset["optimal_move"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Game environment setup
def display_board(board):
    print("  0 1 2 3")  # Column labels
    for i, row in enumerate(board):
        row_str = f"{i} "  # Row label
        for cell in row:
            if cell == 0:
                row_str += ". "  # Empty space
            elif cell == 1:
                row_str += "K "  # White King
            elif cell == 2:
                row_str += "P "  # Black Pawn
        print(row_str)
    print()

def get_king_position(board):
    return np.argwhere(board == 1)[0]

def get_pawn_positions(board):
    return np.argwhere(board == 2)

def model_move(board):
    flattened_board = board.flatten().reshape(1, -1)
    return model.predict(flattened_board)[0]

def player_move(board):
    print("Enter the position of the pawn to move (row and column):")
    pawn_row, pawn_col = map(int, input("Pawn position: ").split())
    print("Enter the new position for the pawn (row and column):")
    new_row, new_col = map(int, input("New position: ").split())

    if board[pawn_row, pawn_col] == 2 and board[new_row, new_col] == 0:
        board[pawn_row, pawn_col] = 0
        board[new_row, new_col] = 2
    else:
        print("Invalid move. Try again.")
        player_move(board)

# Play the game
def play_game():
    board = np.zeros((4, 4), dtype=int)
    board[0, 0] = 1  # White king starts at top-left corner
    board[1, 2] = 2  # Initial black pawns
    board[3, 1] = 2
    board[2, 3] = 2

    print("Game Start!")
    display_board(board)

    for turn in range(20):  # Limit the game to 20 turns
        print(f"Turn {turn + 1}")

        # Model's move
        move = model_move(board)
        king_pos = get_king_position(board)
        king_row, king_col = king_pos
        directions = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1), "stay": (0, 0)}
        dr, dc = directions[move]
        new_king_row, new_king_col = king_row + dr, king_col + dc

        if 0 <= new_king_row < 4 and 0 <= new_king_col < 4:
            board[king_row, king_col] = 0
            board[new_king_row, new_king_col] = 1

        print("White King's move:")
        display_board(board)

        # Check for win/loss conditions
        pawn_positions = get_pawn_positions(board)
        if not pawn_positions.size:
            print("White King wins!")
            break

        if any(np.array_equal(king_pos, pawn_pos) for pawn_pos in pawn_positions):
            print("Black Pawns win!")
            break

        # Player's move
        print("Your turn (Black Pawns):")
        player_move(board)
        display_board(board)

play_game()


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Game Start!
  0 1 2 3
0 K . . . 
1 . . P . 
2 . . . P 
3 . P . . 

Turn 1
White King's move:
  0 1 2 3
0 . . . . 
1 K . P . 
2 . . . P 
3 . P . . 

Your turn (Black Pawns):
Enter the position of the pawn to move (row and column):
Pawn position: 1 2
Enter the new position for the pawn (row and column):
New position: 1 1


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


  0 1 2 3
0 . . . . 
1 K P . . 
2 . . . P 
3 . P . . 

Turn 2
White King's move:
  0 1 2 3
0 K . . . 
1 . P . . 
2 . . . P 
3 . P . . 

Your turn (Black Pawns):
Enter the position of the pawn to move (row and column):
Pawn position: 3 1
Enter the new position for the pawn (row and column):
New position: 2 0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


  0 1 2 3
0 K . . . 
1 . P . . 
2 P . . P 
3 . . . . 

Turn 3
White King's move:
  0 1 2 3
0 . . . . 
1 K P . . 
2 P . . P 
3 . . . . 

Your turn (Black Pawns):
Enter the position of the pawn to move (row and column):
Pawn position: 2 3
Enter the new position for the pawn (row and column):
New position: 2 2


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


  0 1 2 3
0 . . . . 
1 K P . . 
2 P . P . 
3 . . . . 

Turn 4
White King's move:
  0 1 2 3
0 K . . . 
1 . P . . 
2 P . P . 
3 . . . . 

Your turn (Black Pawns):
Enter the position of the pawn to move (row and column):
Pawn position: 1 1
Enter the new position for the pawn (row and column):
New position: 1 0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


  0 1 2 3
0 K . . . 
1 P . . . 
2 P . P . 
3 . . . . 

Turn 5
White King's move:
  0 1 2 3
0 . K . . 
1 P . . . 
2 P . P . 
3 . . . . 

Your turn (Black Pawns):
Enter the position of the pawn to move (row and column):
Pawn position: 1 0
Enter the new position for the pawn (row and column):
New position: 0 1
Invalid move. Try again.
Enter the position of the pawn to move (row and column):
Pawn position: 2 0
Enter the new position for the pawn (row and column):
